# Cross-lingual Learning through Adversarial Training

### Problem Formulation

Let $L_1$ and $L_2$ be two different two natural languages. Let $C_1$ and $C_2$ be two corpora from $L_1$ and $L_2$ repectively.

We denote a document from a document from $C_i$ by $C_i^{(j)}$. Let $Q_i^{(j)}$ be a question based on the information available in the document $C_i^{(j)}$ and Let $A_i^{(j)}$ be the answer to the given question.

We wish to find two encoding networks $\phi_1(\cdot)$, $\phi_2(\cdot)$ and an answerer network $\gamma(\cdot, \cdot)$ such that $\gamma(\phi_i(C_i^{(k)}), \phi_1(Q_i^{(k)})) \to A_i^{(k)}$ for both $i=1$ and $i=2$.

Let $R_i^{(j)}$ be the output of $\phi_i(C_i^{(j)})$ (document representation) and $r_i^{(j)}$ be the output of $\phi_i(Q_i^{(j)})$ (question representation), we hope to minimise the language dependency of $R$ and $r$ so that the answerer network $\gamma$ learns a language-independent way to solve the question-answering task. Ideally, this means that if $C_1^{(j)}$ and $C_2^{(j)}$ are similar in their content, then $||R_1^{(j)} - R_2^{(j)}||$ should be small (same for $r$). In practice, however, it is difficult to measure because:
1. It is unlikely to find two documents in two languages that are perfectly aligned and also suitable for QA tasks;
2. Directly forcing $R_1^{(j)} = R_2^{(j)}$ (i.e. the approach taken by neutral machine translation) leads to a model that is difficult to train.

In order to circumvent the issue of minimising representation difference, we introduce a discriminator network $\delta(\cdot)$ that classifies an input representation $R$ (or $r$) into either "from $L_1$" or "from $L_2$". The goal of $\delta$ is to best distinguish between representations generated from $\phi_1$ and $\phi_2$ (which ultimately means inputs from $L_1$ or $L_2$). We add an additional training goal to $\phi_i$ other than maximising the accuracy of $\gamma$, which is to mimimise the success rate (or maximising the error rate) of $\delta$.

### Details of the Model

$C_i^{(j)}$ is an S by N by V matrix where S is the number of sentences, N is the number of words and V is the size of the vocabulary.

$\phi_i(\cdot)=\phi_i^{summary}(\phi_i^{sent}(\phi_i^{emb}(\cdot)))$

$\phi_i^{emb}$ is the embedding layer, whereas $\phi_i^{sent}$ and $\phi_i^{summary}$ are all bi-RNNs. $\phi_i^{sent}$ generates a single output for each sentence, whereas $\phi_i^{summary}$ generates a context-aware representation for all sentences.

$R_i^{(j)}$ and $r_i^{(j)}$ are S by D and S by 1 matrices, respectively. D is the dimension of the representation.

$\gamma(R, r) = \gamma^{MLP}(r, \sigma_i(R, r) \odot R)$, where $\sigma_i(\cdot, \cdot)$ is a similarity measure (such as dot product) applied across the S dimension of R. $\sigma$ here serves as the attention mechanism. The output of $\gamma$ is a softmax (or logits) vector with the same length as $A_i^{(j)}$. (Problem here: in order to use softmax output, the answer space should be shared between two languages, otherwise the answerer network $\gamma$ cannot achieve language-indenepdence.)

$\delta$ is an MLP that takes either $R_i^{(j)}[k, :]$ or $r_i^{(j)}$ as input and generates a binary output.

### Training Goals

The main goal of the model is to maximise the probability of successfully predicting the answer, i.e.
$$
argmax_{\Theta_\phi, \Theta_\gamma} logP(A|Q, C, \Theta_\phi, \Theta_\gamma)
$$

The auxiliary goal is to maximise the confusion of the discriminator network $\delta$, i.e.
$$
argmax_{\Theta_\phi} log P(\lnot L|Q, C,\Theta_\phi, \Theta_\delta)
$$

The joint goal can be represented as:
$$
argmax_{\Theta_\phi, \Theta_\gamma} [logP(A|Q, C, \Theta_\phi, \Theta_\gamma)
+ \alpha \cdot log P(\lnot L|Q, C,\Theta_\phi, \Theta_\delta)]
$$

Whereas the goal of the discriminator network is to maximise the correct prediction probability, i.e.
$$
argmax_{\Theta_\delta} log P(L|Q, C, \Theta_\phi, \Theta_\delta)
$$

### Analysis of the Problem

In our QA dataset, the query word in the question $Q_i^{(j)}$ is denoted by the marker *@placeholder* and potential answers to the question are the set of anonymised entities denoted by *@entityX* where X is an integer ID. What the answerer network $\gamma^{MLP}(\cdot, \cdot)$ is doing is essentially map a (contextualised) attended entity token back into the entity ID space, so if the entity representations (their token embeddings) are shared between two languages, the network $\gamma$ should not depend on a particular input language to process the mapping, and the task should be able to be completed by a single layer network, i.e. when $\gamma$ is a linear-softmax layer.

We may simplify the model as follows:

$$
\phi_i(\cdot)=\phi_i^{RNN}((\phi_i^{word\_emb}(\cdot) + \phi^{entity\_emb}(\cdot))
$$$$
R_i^{(j)}[:]=\phi_i(C_i^{(j)})[:]
$$$$
r_i^{(j)}=\phi_i(Q_i^{(j)})[-1]
$$$$
\gamma(R, r) = Softmax[Linear(r, \sigma_i(R, r) \odot R)]
$$$$
\delta(\cdot) = \delta^{MLP}(\cdot)
$$

The $\gamma$ layer can be trained with mixed intermediate inputs from $C_1$ and $C_2$, when they share the same entity embeddings $\phi^{entity\_emb}$. However, the bulk of the model, which is the document encoding network $\phi_i^{RNN}$ and similarity measure function $\sigma_i$ (if it is parameterised) are still language-dependent and must be trained with langauge-specific QA pairs.

The goal of adversarial training mentioned above is to remove language-specific information from the attended vector $\sigma_i(R, r) \odot R$. However, it is itself a contextualised representation of a word and is therefore dependent on the language of the document. There are two situations:

1. The word on which attention is calculated is a normal, non-entity word. In this case, the representation $R$ will be language dependent.
2. The word on which attention is calculated is an entity marker. In this case, the representation is a function of its entity ID, which is language-independent, and its context, which is language-dependent. It should be possible to extract the language-independent portion of information from the representation $R$, and map it back to the entity ID.

Therefore, for the adversarial network to work, we should insert an extraction network $\phi^{extract}$ between $R$ and $\gamma$, and only pass the representations of entities through the discriminator network. We could force all of the $R$'s to be language-independent, but that is not so different as to force the word embeddings to be language-independent.

However, this setup means the adversarial training is not doing much. The answerer network is fairly simple, as its only job is to map an attended entity representation back to the entity ID. It might have to perform some "denoising" to discern the entity information from the context that is mixed in by the RNN, but otherwise it is just a reverse embedding lookup. The heavy-lifting steps of the process, i.e. contextualised encoding and attention, still have to be trained per language with potentially a large amount of data for each.

### Possible Alternative Approach

The source of the language-dependency problem is that the two languages use different word embeddings to represent their vocabulary (of course, there are also syntactic and other higher-level differences, but for a simple attention-based model on two similar languages, these are not as significant as the embedding differences. Maybe instead of forcing the contextualised representations to be the same, we can force the word embeddings to be "shared" so that there is no need to use two different set of networks for the task?

Word embeddings cannot be simply shared through establishing one-to-one mapping between words in two different languages, since such mapping does not exist in reality. However, there are some recent research suggesting that it is possible to create a bilingual word embedding in which similar words in two languages are mapped close together, and such an embedding can be created from two separate set of word embeddings and a small set of equivalent words.


>Mikel Artetxe, Gorka Labaka, and Eneko Agirre. 2016. Learning principled bilingual mappings of word embeddings while preserving monolingual invariance. In Proceedings of the 2016 Conference on Empirical Methods in Natural Language Processing, pages 2289-2294.

>Mikel Artetxe, Gorka Labaka, and Eneko Agirre. 2017. Learning bilingual word embeddings with (almost) no bilingual data. In Proceedings of the 55th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers), pages 451-462.

If we can use a shared embedding between $L_1$ and $L_2$, we can further simplify the model to use completely shared network layers:

$$
\phi(\cdot)=\phi^{RNN}((\phi^{word\_emb_{L_1, L_2}}(\cdot) + \phi^{entity\_emb}(\cdot))
$$$$
R_i^{(j)}[:]=\phi(C_i^{(j)})[:]
$$$$
r_i^{(j)}=\phi(Q_i^{(j)})[-1]
$$$$
\gamma(R, r) = Softmax[Linear(r, \sigma(R, r) \odot R)]
$$

# TO BE DISCUSSED

1. Role of final layer
2. Adversarial vs shared embeddings
3. Performance on original CNN data vs reported performance

### Issues with the Data (DONE)

In order to train a cross-lingual model on question answering, we need decent quality QA corpus in more than one language. Ideally, the length, topic etc. of the two corpora should align as well, otherwise the discriminator network will likely be able to distinguish two languages by these non-language-specific features.

So far, there is a lack of datasets that enable cross-lingual training of question answering models. Below are some of the possible sources of QA datasets:

1. Parallel texts. Plenty of parallel texts exist, and they are often of the exact same topics. Some of them are even aligned to the sentence or word level. These attributes are desirable in our model training. However, these corpora are usually only compiled for translation purposes, and many of them are not suitable for conversion into QA datasets.

2. Existing QA datasets in two or more languages. There might be issues with the topic / vocabulary alignment of datasets, but this is the most convenient approach.

3. Compile a new dataset from web data etc. so that it is compatible with an existing QA dataset.

Here are some possible datasets for use:

1. The Stanford Question Answering Dataset - https://rajpurkar.github.io/SQuAD-explorer/ High quality QA dataset. Based on Wikipedia.

2. WebQA: A Chinese Open-Domain Factoid Question Answering Dataset - http://idl.baidu.com/WebQA.html. Similar idea to SQUAD, however different in various ways (based on community QA websites, contains possibly useless evidence, informal language)

3. OPUS - http://opus.lingfil.uu.se/index.php Large amount of parallel corpora. However challenging to generate questions from these documents. Many of them are fragmented (only aligned sentences are retained)

4. LDC - (example) https://catalog.ldc.upenn.edu/LDC95T13 High data usage fee

5. Select one well-known corpus in multiple languages, such as http://www.umiacs.umd.edu/~resnik/parallel/bible.html Relatively easy to find existing Q-A pairs from other sources and put them togetehr as a dataset. E.g. There are trivia questions on http://www.christianity.com/trivia/category/. However extracting the answers still require some effort.

**Chosen Solution**: compile our own news QA dataset from news website scrapes.